In [1]:
import os
import pandas as pd
import numpy as np

#-------------------Inputs-------------------------------------------------------------
input_file = "Shear_and_Moment_6 Paddlers.csv" # or "Shear_and_Moment_4 Paddlers.csv" or "Shear_and_Moment_2 Paddlers.csv"
hf = 25 # [mm]
f_c = 10 # [MPa]
lamda = 0.75
beta = 0.21
layer1 = 1 # if layer exists, it equals to 1. Otherwise, zero.
layer2 = 0 # if layer exists, it equals to 1. Otherwise, zero.
d_mesh_1 = 12.5 # distance b/w the first layer of reinforcement to the surface [mm]
d_mesh_2 = 0 # distance b/w the second layer of reinforcement to the surface [mm]
#-----------Reinforcement Property-------------
fi_s = 0.9 
fy = 4347 # [MPa]
fi_c = 0.65
E_s = 240000 # [MPa]
layer_unit_1 = 46 # one lap for layer 1 [mm^2/m]
layer_unit_2 = 0 # one lap for layer 2
#--------------------------------------------------------------------------------------

script_dir = os.path.abspath('')
file_path = os.path.join(script_dir, input_file)
info = pd.read_csv(file_path, header=None)


In [2]:
# Positive Shear
value = float(info[1][1])
width = float(info[3][1])
depth = float(info[4][1])
bw = float(info[5][1])

h = depth

if layer2 == 0:
    d = depth - hf + d_mesh_1
else:
    d = depth - hf + (d_mesh_1+d_mesh_2)/2

dv = max(0.9*d, 0.72*h)

Vr_max = 0.25*fi_c*f_c*bw*dv
Vc = fi_c*lamda*beta*(min(f_c,64))*bw*dv

print('Positive Shear: Vr_max is  ', Vr_max, 'N.')
print('Positive Shear: Vc is  ', Vc, 'N.')


Positive Shear: Vr_max is   46595.294831399704 N.
Positive Shear: Vc is   29355.03574378182 N.


In [3]:
# Negative Shear
value = float(info[1][2])
width = float(info[3][2])
depth = float(info[4][2])
bw = float(info[5][2])

h = depth

if layer2 == 0:
    d = depth - hf + d_mesh_1
else:
    d = depth - hf + (d_mesh_1+d_mesh_2)/2

dv = max(0.9*d, 0.72*h)

Vr_max = 0.25*fi_c*f_c*bw*dv
Vc = fi_c*lamda*beta*(min(f_c,64))*bw*dv

print('Negative Shear: Vr_max is  ', Vr_max, 'N.')
print('Negative Shear: Vc is  ', Vc, 'N.')


Negative Shear: Vr_max is   52588.11729796278 N.
Negative Shear: Vc is   33130.513897716555 N.
